In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from torch import optim, nn

from plotting import plot_model_outputs

import beer

output_notebook()

Loading BokehJS ...

In [2]:
mean = np.array([3., 3.])
cov = np.array([[2., 1.5], [1.5, 2.]])
data = np.random.multivariate_normal(mean, cov, size=100)

fig = figure(
    title='Linear subspace',
    width=400,
    height=400,
)
fig.circle(data[:, 0], data[:, 1])
show(fig)

In [3]:
obs_dim = 2
latent_dim = 2
nb_samples = 10

enc_struct = nn.Sequential(
    nn.Linear(obs_dim, 2),
    nn.Tanh(),
    nn.Linear(2, 2),
    nn.Tanh()
)
encoder = beer.models.MLPNormalDiag(enc_struct, 2, latent_dim)

dec_struct = nn.Sequential(
    nn.Linear(latent_dim, 2),
    nn.Tanh(),
    nn.Linear(2, 2),
    nn.Tanh()
)
decoder = beer.models.MLPNormalIso(dec_struct, 2, obs_dim)

#latent_model = beer.models.NormalDiagonalCovariance.create(
#    latent_dim,
#    mean_prec=1e2,
#    prec_shape=1e-3,
#    prec_rate=1e-3
#)
latent_model = beer.models.NaturalIsotropicGaussian(2)

model = beer.models.VAE(encoder, decoder, latent_model, nb_samples)

In [4]:
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
history = beer.inference.History(report_interval=250)

In [5]:
beer.inference.run_training(data, model, optimizer, 2000, history, batch_size=20, lrate_latent_model=1e-1)

Epoch: 250 	elbo: -21.305396 llh: -420.004402 kld: 6.103528
Epoch: 500 	elbo: -17.677380 llh: -353.315445 kld: 0.232163
Epoch: 750 	elbo: -17.607800 llh: -349.627792 kld: 2.528211
Epoch: 1000 	elbo: -17.375670 llh: -332.829685 kld: 14.683718
Epoch: 1250 	elbo: -16.528929 llh: -278.429562 kld: 52.149003
Epoch: 1500 	elbo: -16.415235 llh: -269.104065 kld: 59.200636
Epoch: 1750 	elbo: -16.275213 llh: -263.623692 kld: 61.880581
Epoch: 2000 	elbo: -16.564226 llh: -271.324455 kld: 59.960074


In [6]:
plot_model_outputs(model, data)

In [7]:
history.plot()

In [96]:
print(model.latent_model.exp_mean)
print(1/model.latent_model.exp_precision)
print(model.latent_model.exp_log_precision)

AttributeError: 'NaturalIsotropicGaussian' object has no attribute 'exp_mean'

In [ ]:
np.log(181)